In [ ]:
# default_exp importing

# Data import

## This notebook provides a function to import peptide level data from Spectronaut, MaxQuant, AlphaPept, DIA-NN

The preprocessed data is stored in a pandas dataframe with following columns:
* all_protein_ids: all UniProt IDs the peptide map to separated by ';'
* modified_sequence: the peptide sequence with all modifications included in square brackets
* naked_sequence: the naked peptide sequence

It is possible to further select one or more specific samples for import. A single sample can be provided as character string. Multiple samples can be provided as list of character strings. The raw MS filename should match corresponding entries in the "R.FileName", "Raw file", "shortname" or "Run" column of the Spectronaut, MaxQuant, AlphaPept or DIA-NN analysis respectively.

In [ ]:
#export
import os
import pandas as pd

def read_file(
    file: str,
    column_names: list
) -> pd.DataFrame:
    """Load a specified columns of the file as a pandas dataframe.

    Args:
        file (str): The name of a file.
        column_names (list): The list of three columns that should be extracted from the file.
    
    Raises:
        NotImplementedError: if a specified file has not a .csv, .txt or .tsv extension.
        ValueError: if any of the specified columns is not in the file.
    
    Returns:
        pd.DataFrame: A pandas dataframe with all the data stored in the specified columns.
    """
    file_ext = os.path.splitext(file)[-1]
    if file_ext=='.csv':
        sep=','
    elif file_ext=='.tsv':
        sep='\t'
    elif file_ext=='.txt':
        sep='\t'
    else:
        raise NotImplementedError("The selected filetype isn't supported. Please specify a file with a .csv, .txt or .tsv extension.")
    with open(file) as filelines:
        i = 0
        pos = 0
        for l in filelines:
            i += 1
            l = l.split(sep)
            try:
                raw = l.index(column_names[0])
                prot = l.index(column_names[1])
                seq = l.index(column_names[2])
            except:
                raise ValueError('The list of specified column names cannot be extracted from the file.')
            if i>0:
                break

    with open(file) as filelines:
        raws = []
        prots = []
        seqs = []
        for l in filelines:
            l = l.split(sep)
            raws.append(l[raw])
            prots.append(l[prot])
            seqs.append(l[seq])

    res = pd.DataFrame({column_names[0]:raws[1:],
             column_names[1]:prots[1:],
             column_names[2]:seqs[1:]})

    return res


def extract_rawfile_unique_values(
    file: str
) -> list:
    """Extract the unique raw file names from "R.FileName" (Spectronaut output), "Raw file" (MaxQuant output),
    "shortname" (AlphaPept output) or "Run" (DIA-NN output) column.

    Args:
        file (str): The name of a file.
    
    Raises:
        ValueError: if a column with the unique raw file names is not in the file.
    
    Returns:
        list: A sorted list of unique raw file names from the file.
    """
    file_ext = os.path.splitext(file)[-1]
    if file_ext == '.csv':
        sep = ','
    elif file_ext in ['.tsv', '.txt']:
        sep = '\t'

    with open(file) as filelines:
        i = 0
        filename_col_index = None
        filename_data = []

        for l in filelines:
            l = l.split(sep)
            # just do it for the first line
            if i == 0:
                for col in ['R.FileName', 'Raw file', 'Run', 'shortname']:
                    if col in l:
                        filename_col_index = l.index(col)
                        break      
                if not isinstance(filename_col_index, int):
                    raise ValueError('A column with the raw file names is not in the file.')
            else:
                filename_data.append(l[filename_col_index])
            i += 1

        unique_filenames = set(filename_data)

    sorted_unique_filenames = sorted(list(unique_filenames))
    return sorted_unique_filenames

In [ ]:
#hide
def test_read_file():
    file_with_missing_col = '../testdata/test_not_all_columns_spectronaut.csv'
    spectronaut_columns = ["PEP.AllOccurringProteinAccessions","EG.ModifiedSequence","R.FileName"]
    
    try:
        out = read_file(file_with_missing_col, spectronaut_columns)
    except ValueError as e:
        out = e
    assert str(out) == "The list of specified column names cannot be extracted from the file." 

def test_extract_rawfile_unique_values():
    data_Spectronaut_csv = "../testdata/test_spectronaut_input.csv"
    data_Spectronaut_tsv = "../testdata/test_spectronaut_input.tsv"
    assert ['raw_01', 'raw_02'] == extract_rawfile_unique_values(data_Spectronaut_csv)
    assert extract_rawfile_unique_values(data_Spectronaut_csv) == extract_rawfile_unique_values(data_Spectronaut_tsv)
    
    data_MQ = "../testdata/test_maxquant_input.txt"
    assert len(extract_rawfile_unique_values(data_MQ)) == 381
    
    data_wrong = '../testdata/test_maxquant_imported.csv'
    try:
        out = extract_rawfile_unique_values(data_wrong)
    except ValueError as e:
        out = e
    assert str(out) == "A column with the raw file names is not in the file." 

test_read_file()
test_extract_rawfile_unique_values()

## Import Spectronaut data

In [ ]:
#export
import pandas as pd
import re
from typing import Union

def import_spectronaut_data(
    file: str,
    sample: Union[str, list, None] = None
) -> pd.DataFrame:
    """Import peptide level data from Spectronaut.

    Args:
        file (str): The name of a file.
        sample (Union[str, list, None]): The unique raw file name(s) to filter the original file. Defaults to None. In this case data for all raw files will be extracted.

    Returns:
        pd.DataFrame: A pandas dataframe containing information about: all_protein_ids (str), modified_sequence (str), naked_sequence (str)
    """
    spectronaut_columns = ["PEP.AllOccurringProteinAccessions","EG.ModifiedSequence","R.FileName"]

    data = read_file(file, spectronaut_columns)

    if sample:
        if isinstance(sample, list):
            data_sub = data[data["R.FileName"].isin(sample)]
            data_sub = data_sub[["PEP.AllOccurringProteinAccessions","EG.ModifiedSequence"]]
        elif isinstance(sample, str):
            data_sub = data[data["R.FileName"] == sample]
            data_sub = data_sub[["PEP.AllOccurringProteinAccessions","EG.ModifiedSequence"]]
    else:
        data_sub = data[["PEP.AllOccurringProteinAccessions","EG.ModifiedSequence"]]

    # get modified sequence
    mod_seq = data_sub.apply(lambda row: re.sub('_','',row["EG.ModifiedSequence"]), axis=1)
    data_sub = data_sub.assign(modified_sequence=mod_seq.values)
    # get naked sequence
    nak_seq = data_sub.apply(lambda row: re.sub(r'\[.*?\]','',row["modified_sequence"]), axis=1)
    data_sub = data_sub.assign(naked_sequence=nak_seq.values)
    data_sub = data_sub.rename(columns={"PEP.AllOccurringProteinAccessions": "all_protein_ids"})
    input_data = data_sub[["all_protein_ids","modified_sequence","naked_sequence"]]
    input_data = input_data.dropna()
    input_data = input_data.drop_duplicates().reset_index(drop=True)
    return input_data

In [ ]:
#hide

def test_import_spectronaut_data():
    # test entire input test data
    data = import_spectronaut_data("../testdata/test_spectronaut_input.csv")
    #print(data.shape[0])
    assert data.shape[0] == 40
    data_t = import_spectronaut_data("../testdata/test_spectronaut_input.tsv")
    #print(data_t.shape[0])
    pd.testing.assert_frame_equal(data, data_t)
    test = pd.read_csv('../testdata/test_spectronaut_imported.csv', sep=',') 
    #print(test.shape[0])
    pd.testing.assert_frame_equal(data, test)
    
    # test single sample
    data = import_spectronaut_data("../testdata/test_spectronaut_input.csv", 
                                   sample="raw_01")
    #print(data.shape[0])
    assert data.shape[0] == 40
    data = import_spectronaut_data("../testdata/test_spectronaut_input.csv", 
                                   sample="raw_02")
    #print(data.shape[0])
    assert data.shape[0] == 20
    
    # test multiple samples
    data = import_spectronaut_data("../testdata/test_spectronaut_input.csv", 
                                   sample=["raw_01","raw_02"])
    #print(data.shape[0])
    assert data.shape[0] == 40

test_import_spectronaut_data()

## Import MaxQuant data

In [ ]:
#export
import pandas as pd
from typing import Union
import re

def import_maxquant_data(
    file: str,
    sample: Union[str, list, None] = None
) -> pd.DataFrame:
    """Import peptide level data from MaxQuant.

    Args:
        file (str): The name of a file.
        sample (Union[str, list, None]): The unique raw file name(s) to filter the original file. Defaults to None. In this case data for all raw files will be extracted.

    Returns:
        pd.DataFrame: A pandas dataframe containing information about: all_protein_ids (str), modified_sequence (str), naked_sequence (str)
    """
    mq_columns = ["Proteins","Modified sequence","Raw file"]

    data = read_file(file, mq_columns)

    if sample:
        if isinstance(sample, list):
            data_sub = data[data["Raw file"].isin(sample)]
            data_sub = data_sub[["Proteins","Modified sequence"]]
        elif isinstance(sample, str):
            data_sub = data[data["Raw file"] == sample]
            data_sub = data_sub[["Proteins","Modified sequence"]]
    else:
        data_sub = data[["Proteins","Modified sequence"]]

    # get modified sequence
    mod_seq = data_sub.apply(lambda row: re.sub('_','',row["Modified sequence"]), axis=1)
    data_sub = data_sub.assign(modified_sequence=mod_seq.values)

    # replace outer () with []
    mod_seq_replaced = data_sub.apply(lambda row: re.sub(r'\((.*?\(.*?\))\)',r'[\1]',row["modified_sequence"]), axis=1)
    data_sub = data_sub.assign(modified_sequence=mod_seq_replaced.values)

    # get naked sequence
    nak_seq = data_sub.apply(lambda row: re.sub(r'\[.*?\]','',row["modified_sequence"]), axis=1)
    data_sub = data_sub.assign(naked_sequence=nak_seq.values)
    data_sub = data_sub.rename(columns={"Proteins": "all_protein_ids"})
    input_data = data_sub[["all_protein_ids","modified_sequence","naked_sequence"]]
    input_data = input_data.dropna() # remove missing values
    input_data = input_data.drop_duplicates().reset_index(drop=True)
    return input_data

In [ ]:
#hide

def test_import_maxquant_data():
    data = import_maxquant_data("../testdata/test_maxquant_input.txt")
    test = pd.read_csv('../testdata/test_maxquant_imported.csv', sep=',') 
    pd.testing.assert_frame_equal(data, test)
    
    data_s = import_maxquant_data("../testdata/test_maxquant_input.txt", 
                                  sample = "raw_1")
    assert data_s.shape[0] == 85
    
    data_s = import_maxquant_data("../testdata/test_maxquant_input.txt", 
                                  sample = "raw_2")
    assert data_s.shape[0] == 77
    
    data_s = import_maxquant_data("../testdata/test_maxquant_input.txt", 
                                  sample = ["raw_1", "raw_2"])
    assert data_s.shape[0] == 136

test_import_maxquant_data()

## Import AlphaPept data

In [ ]:
#export
import re

def convert_ap_mq_mod(
    sequence:str
) -> str:
    """Convert AlphaPept style modifications into MaxQuant style modifications.

    Args:
        sequence (str): The peptide sequence with modification in an AlphaPept style.

    Returns:
        str: The peptide sequence with modification in a similar to MaxQuant style.
    """
    # TODO: add more AP modifications
    modif_convers_dict = {
        'ox': '[Oxidation ({})]',
        'a': '[Acetyl ({})]',
        'am': '[Amidated ({})]',
        'deam': '[Deamidation ({})]',
        'p': '[Phospho ({})]',
        'pg': '[{}->pyro-Glu]',
        'c': '[Cys-Cys]'
    }
    mods = re.findall('[a-z0-9]+', sequence)
    if mods:
        for mod in mods:
            posit = re.search(mod, sequence)
            i = posit.start()
            if i == 0 and mod == 'a':
                add_aa = 'N-term'
            elif posit.end() == len(sequence) - 1 and mod == 'am':
                add_aa = sequence[posit.end()]
                sequence = sequence.replace(mod + add_aa, add_aa + mod, 1)
                add_aa = 'C-term'
            else:
                add_aa = sequence[posit.end()]
                sequence = sequence.replace(mod + add_aa, add_aa + mod, 1)

            if mod == 'ox':
                if add_aa == 'M':
                    add_aa = 'M'
                elif add_aa in 'MP':
                    add_aa = 'MP'
            elif mod == 'deam':
                if add_aa in 'NQ':
                    add_aa = 'NQ'
            elif mod == 'p':
                if add_aa in 'STY':
                    add_aa = 'STY'
            elif mod == 'pg':
                if add_aa == 'E':
                    add_aa = 'Glu'
                elif add_aa == 'Q':
                    add_aa = 'Gln'

            if mod in modif_convers_dict.keys():
                sequence = sequence.replace(mod, modif_convers_dict.get(mod).format(add_aa), 1)
    return sequence

In [ ]:
#hide

def test_convert_ap_mq_mod():
    # test oxidation
    seq1 = 'HAEoxMVHTGLK' 
    assert "HAEM[Oxidation (M)]VHTGLK" == convert_ap_mq_mod(seq1)
    seq2 = 'HAEoxPVHTGLK' 
    assert "HAEP[Oxidation (MP)]VHTGLK" == convert_ap_mq_mod(seq2)
    # test acetylation
    seq4 = 'aMDEPSPLAQPLELNQHSR' 
    assert "[Acetyl (N-term)]MDEPSPLAQPLELNQHSR" == convert_ap_mq_mod(seq4)
    seq5 = 'MDEPSaKPLA' 
    assert "MDEPSK[Acetyl (K)]PLA" == convert_ap_mq_mod(seq5)
    # test amidation
    seq6 = 'MDEPSKPLamA' 
    assert "MDEPSKPLA[Amidated (C-term)]" == convert_ap_mq_mod(seq6)
    # test deamidation
    seq7 = 'MDEPSdeamNKPLA' 
    assert "MDEPSN[Deamidation (NQ)]KPLA" == convert_ap_mq_mod(seq7)
    seq8 = 'MDEPSdeamQKPLA' 
    assert "MDEPSQ[Deamidation (NQ)]KPLA" == convert_ap_mq_mod(seq8)
    # test phosporylation
    seq9 = 'MDEPSpSKPLA' 
    assert "MDEPSS[Phospho (STY)]KPLA" == convert_ap_mq_mod(seq9)
    # test pyro-Glu
    seq10 = 'MDpgEPSNKPLA' 
    assert "MDE[Glu->pyro-Glu]PSNKPLA" == convert_ap_mq_mod(seq10)
    seq11 = 'MDpgQPSNKPLA' 
    assert "MDQ[Gln->pyro-Glu]PSNKPLA" == convert_ap_mq_mod(seq11)
    # test disylfide bonds
    seq12 = 'cCVNTTLQIK'
    assert "C[Cys-Cys]VNTTLQIK" == convert_ap_mq_mod(seq12)
    
    seq1_several_dif_mods = 'AcCLDYPVTSVLPPASLoxMK'
    assert "AC[Cys-Cys]LDYPVTSVLPPASLM[Oxidation (M)]K" == convert_ap_mq_mod(seq1_several_dif_mods)
    seq2_several_same_mods = 'LFTToxMELoxMR'
    assert "LFTTM[Oxidation (M)]ELM[Oxidation (M)]R" == convert_ap_mq_mod(seq2_several_same_mods)
    seq3_several_same_mods = 'LFTToxMELoxMRoxM'
    assert "LFTTM[Oxidation (M)]ELM[Oxidation (M)]RM[Oxidation (M)]" == convert_ap_mq_mod(seq3_several_same_mods)
    seq4_several_same_mods = 'LFTTdeamNELdeamNR'
    assert "LFTTN[Deamidation (NQ)]ELN[Deamidation (NQ)]R" == convert_ap_mq_mod(seq4_several_same_mods)
    seq_no_mod = 'CVNTTLQIK'
    assert "CVNTTLQIK" == convert_ap_mq_mod(seq_no_mod)
    
test_convert_ap_mq_mod()

In [ ]:
#export
import pandas as pd
from typing import Union

def import_alphapept_data(
    file: str,
    sample: Union[str, list, None] = None
) -> pd.DataFrame:
    """Import peptide level data from AlphaPept.

    Args:
        file (str): The name of a file.
        sample (Union[str, list, None]): The unique raw file name(s) to filter the original file. Defaults to None. In this case data for all raw files will be extracted.

    Returns:
        pd.DataFrame: A pandas dataframe containing information about: all_protein_ids (str), modified_sequence (str), naked_sequence (str)
    """
    ap_columns = ["protein_group", "sequence", "shortname"]

    data = pd.read_csv(file, usecols=ap_columns)
    # TODO: add later the file reading using read_file function. For now it doesn't work for the protein groups that should be split later

    if sample:
        if isinstance(sample, list):
            data_sub = data[data["shortname"].isin(sample)]
            data_sub = data_sub[["protein_group", "sequence"]]
        elif isinstance(sample, str):
            data_sub = data[data["shortname"] == sample]
            data_sub = data_sub[["protein_group", "sequence"]]
    else:
        data_sub = data[["protein_group", "sequence"]]

    data_sub = data_sub[~data_sub.sequence.str.contains('_decoy')]

    # get modified sequence
    modif_seq = data_sub.apply(lambda row: convert_ap_mq_mod(row.sequence), axis=1)
    data_sub['modified_sequence'] = modif_seq.values

    # get a list of proteins_id
    proteins = data_sub.apply(lambda row: ";".join([_.split('|')[1] for _ in row.protein_group.split(',')]), axis=1)
    data_sub['all_protein_ids'] = proteins.values

    # get naked sequence
    nak_seq = data_sub.apply(lambda row: ''.join([_ for _ in row.sequence if _.isupper()]), axis=1)
    data_sub['naked_sequence'] = nak_seq.values

    input_data = data_sub[["all_protein_ids", "modified_sequence", "naked_sequence"]]
    input_data = input_data.dropna() # remove missing values
    input_data = input_data.drop_duplicates().reset_index(drop=True)
    return input_data

In [ ]:
#hide

def test_import_alphapept_data():
    data = import_alphapept_data("../testdata/test_alphapept_input.csv")
    assert data.shape == (4228, 3)
    
    data_s1 = import_alphapept_data("../testdata/test_alphapept_input.csv", 
                                  sample = "exp_1")
    assert data_s1.shape[0] == 2127
    
    data_s2 = import_alphapept_data("../testdata/test_alphapept_input.csv", 
                                  sample = "exp_2")
    assert data_s2.shape[0] == 2101
    
    data_s_both = import_alphapept_data("../testdata/test_alphapept_input.csv", 
                                  sample = ["exp_1", "exp_2"])
    assert data_s_both.shape[0] == 4228

test_import_alphapept_data()

## Import DIA-NN data

In [ ]:
#export
import re

def convert_diann_mq_mod(
    sequence:str
) -> str:
    """Convert DIA-NN style modifications into MaxQuant style modifications.

    Args:
        sequence (str): The peptide sequence with modification in an AlphaPept style.

    Returns:
        str: The peptide sequence with modification in a similar to DIA-NN style.
    """

    modif_convers_dict = {
        '(UniMod:1)': '[Acetyl ({})]',
        '(UniMod:2)': '[Amidated ({})]',
        '(UniMod:4)': '[Carbamidomethyl ({})]',
        '(UniMod:5)': '[Carbamyl ({})]',
        '(UniMod:7)': '[Deamidation ({})]',
        '(UniMod:21)': '[Phospho ({})]',
        '(UniMod:23)': '[Dehydrated ({})]',
        '(UniMod:26)': '[Pyro-carbamidomethyl ({})]',
        '(UniMod:27)': '[Glu->pyro-Glu]',
        '(UniMod:28)': '[Gln->pyro-Glu]',
        '(UniMod:30)': '[Cation:Na ({})]',
        '(UniMod:34)': '[Methyl ({})]',
        '(UniMod:35)': '[Oxidation ({})]',
        '(UniMod:36)': '[Dimethyl ({})]',
        '(UniMod:37)': '[Trimethyl ({})]',
        '(UniMod:40)': '[Sulfo ({})]',
        '(UniMod:55)': '[Cys-Cys]',
        '(UniMod:121)': '[GlyGly ({})]',
        '(UniMod:254)': '[Delta:H(2)C(2) ({})]',
        '(UniMod:312)': '[Cysteinyl]',
        '(UniMod:345)': '[Trioxidation ({})]',
        '(UniMod:408)': '[Hydroxyproline]',
        '(UniMod:425)': '[Dioxidation ({})]',
        '(UniMod:526)': '[Dethiomethyl ({})]',
        '(UniMod:877)': '[QQTGG ({})]',
    }
    mods = re.findall('\(UniMod:\d+\)', sequence)
    if mods:
        for mod in mods:
            posit = re.search('\(UniMod:\d+\)', sequence)
            i = posit.start()

            if i == 0:
                add_aa = 'N-term'
            elif posit.end() == len(sequence):
                add_aa = 'C-term'
            else:
                add_aa = sequence[i-1]

            if mod == '(UniMod:7)':
                if add_aa in 'NQ':
                    add_aa = 'NQ'
            elif mod == '(UniMod:21)':
                if add_aa in 'STY':
                    add_aa = 'STY'
            elif mod == '(UniMod:23)':
                if add_aa in 'ST':
                    add_aa = 'ST'
            elif mod == '(UniMod:30)':
                if add_aa in 'DE':
                    add_aa = 'DE'
            elif mod == '(UniMod:34)':
                if add_aa in 'KR':
                    add_aa = 'KR'
            elif mod == '(UniMod:36)':
                if add_aa in 'KR':
                    add_aa = 'KR'
            elif mod == '(UniMod:40)':
                if add_aa in 'STY':
                    add_aa = 'STY'
            elif mod == '(UniMod:425)':
                if add_aa in 'MW':
                    add_aa = 'MW'

            if mod in modif_convers_dict.keys():
                sequence = sequence.replace(mod, modif_convers_dict.get(mod).format(add_aa), 1)

    return sequence

In [ ]:
#hide

def test_convert_diann_mq_mod():
    seq1 = 'VSHGSSPSLLEALSSDFLAC(UniMod:4)K'
    assert 'VSHGSSPSLLEALSSDFLAC[Carbamidomethyl (C)]K' == convert_diann_mq_mod(seq1)
    seq2 = 'VSVINTVDTSHEDMIHDAQM(UniMod:35)DYYGTR'
    assert 'VSVINTVDTSHEDMIHDAQM[Oxidation (M)]DYYGTR' == convert_diann_mq_mod(seq2)
    seq3 = 'HAEMPVHTGLK(UniMod:2)' 
    assert 'HAEMPVHTGLK[Amidated (C-term)]' == convert_diann_mq_mod(seq3)
    seq4 = 'HAEMPVHTGLKS(UniMod:23)A'
    assert 'HAEMPVHTGLKS[Dehydrated (ST)]A' == convert_diann_mq_mod(seq4)
    seq5 = 'HAEMPVHTGLKY(UniMod:23)A' 
    assert 'HAEMPVHTGLKY[Dehydrated (Y)]A' == convert_diann_mq_mod(seq5)
    
    seq1_several_dif_mods = '(UniMod:1)VSHGSSPSLLEALSSDFLAC(UniMod:4)K'
    assert '[Acetyl (N-term)]VSHGSSPSLLEALSSDFLAC[Carbamidomethyl (C)]K' == convert_diann_mq_mod(seq1_several_dif_mods)
    seq2_several_same_mods = 'CAALVATAEENLC(UniMod:4)C(UniMod:4)EELSSK'
    assert 'CAALVATAEENLC[Carbamidomethyl (C)]C[Carbamidomethyl (C)]EELSSK' == convert_diann_mq_mod(seq2_several_same_mods)
    seq_no_mod = 'CVNTTLQIK'
    assert "CVNTTLQIK" == convert_diann_mq_mod(seq_no_mod)
    
test_convert_diann_mq_mod()

In [ ]:
#export
import pandas as pd
from typing import Union

def import_diann_data(
    file: str,
    sample: Union[str, list, None] = None
) -> pd.DataFrame:
    """Import peptide level data from DIA-NN.

    Args:
        file (str): The name of a file.
        sample (Union[str, list, None]): The unique raw file name(s) to filter the original file. Defaults to None. In this case data for all raw files will be extracted.

    Returns:
        pd.DataFrame: A pandas dataframe containing information about: all_protein_ids (str), modified_sequence (str), naked_sequence (str)
    """
    diann_columns = ["Protein.Ids", "Modified.Sequence", "Run"]

    data = read_file(file, diann_columns)

    if sample:
        if isinstance(sample, list):
            data_sub = data[data["Run"].isin(sample)]
            data_sub = data_sub[["Protein.Ids", "Modified.Sequence"]]
        elif isinstance(sample, str):
            data_sub = data[data["Run"] == sample]
            data_sub = data_sub[["Protein.Ids", "Modified.Sequence"]]
    else:
        data_sub = data[["Protein.Ids", "Modified.Sequence"]]

    # get a list of proteins_id
    data_sub = data_sub.rename(columns={"Protein.Ids": "all_protein_ids"})

    # get modified sequence
    modif_seq = data_sub.apply(lambda row: convert_diann_mq_mod(row["Modified.Sequence"]), axis=1)
    data_sub['modified_sequence'] = modif_seq.values

    # get naked sequence
    nak_seq = data_sub.apply(lambda row: re.sub(r'\[.*?\]', '', row["modified_sequence"]), axis=1)
    data_sub = data_sub.assign(naked_sequence = nak_seq.values)

    input_data = data_sub[["all_protein_ids", "modified_sequence", "naked_sequence"]]
    input_data = input_data.dropna() # remove missing values
    input_data = input_data.drop_duplicates().reset_index(drop=True)
    return input_data

In [ ]:
#hide
def test_import_diann_data():
    data = import_diann_data("../testdata/test_diann_input.tsv")
    assert data.shape == (44, 3)
    
    data_s1 = import_diann_data("../testdata/test_diann_input.tsv", 
                                  sample = "20201218_tims03_Evo03_PS_SA_HeLa_200ng_high_speed_21min_8cm_S2-B2_1_22648")
    assert data_s1.shape[0] == 39
    
    data_s_both = import_diann_data("../testdata/test_diann_input.tsv", 
                                  sample = ["20201218_tims03_Evo03_PS_SA_HeLa_200ng_high_speed_21min_8cm_S2-B2_1_22648", 
                                            "20201218_tims03_Evo03_PS_SA_HeLa_200ng_high_speed_21min_8cm_S2-A2_1_22636"])
    assert data_s_both.shape[0] == 42
    
test_import_diann_data()

## Import FragPipe/MSFragger data

In [106]:
#export
import re

def convert_fragpipe_mq_mod(
    sequence:str,
    assigned_modifications: str
) -> str:
    """Convert FragPipe style modifications into MaxQuant style modifications.

    Args:
        sequence (str): The peptide sequence with modification.
        assigned_modifications (str): The string of assigned modifications separated by comma.

    Returns:
        str: The peptide sequence with modification in a similar to DIA-NN style.
    """

    modif_convers_dict = {
        42.0106: '[Acetyl ({})]',
        -0.9840: '[Amidated ({})]',
        57.0215: '[Carbamidomethyl ({})]',
        43.0058: '[Carbamyl ({})]',
        0.9840: '[Deamidation ({})]',
        79.9663: '[Phospho ({})]',
        -18.0106: ['[Dehydrated ({})]', '[Glu->pyro-Glu]'],
        39.9949: '[Pyro-carbamidomethyl ({})]',
        -17.0265: '[Gln->pyro-Glu]',
        21.9819: '[Cation:Na ({})]',
        14.0157: '[Methyl ({})]',
        15.9949: '[Oxidation ({})]',
        28.0313: '[Dimethyl ({})]',
        42.047: '[Trimethyl ({})]',
        79.9568: '[Sulfo ({})]',
        305.0682: '[Cys-Cys]',
        114.0429: '[GlyGly ({})]',
        26.0157: '[Delta:H(2)C(2) ({})]',
        119.0041: '[Cysteinyl]',
        47.9847: '[Trioxidation ({})]',
        148.0372: '[Hydroxyproline]',
        31.9898: '[Dioxidation ({})]',
        -48.0034: '[Dethiomethyl ({})]',
        599.2663: '[QQTGG ({})]',
    }
    
    if assigned_modifications:
        modifs_posit = [''] * (len(sequence) + 1)
        for mod in assigned_modifications.split(','):
            mod = mod.strip()
            data = mod.replace(')', '').split('(')
            mod_pos, mod_mass = data[0], float(data[1])
            if mod_pos == 'N-term':
                posit = 0
                add_aa = 'N-term'
            elif mod_pos == 'C-term':
                posit = -1
                add_aa = 'C-term'
            else:    
                posit = int(mod_pos[:-1])
                add_aa = mod_pos[-1]
                if mod_mass == 0.9840:
                    if add_aa in 'NQ':
                        add_aa = 'NQ'
                elif mod_mass == 79.9663:
                    if add_aa in 'STY':
                        add_aa = 'STY'
                elif mod_mass == 21.9819:
                    if add_aa in 'DE':
                        add_aa = 'DE'
                elif mod_mass == 14.0157:
                    if add_aa in 'KR':
                        add_aa = 'KR'
                elif mod_mass == 28.0313:
                    if add_aa in 'KR':
                        add_aa = 'KR'
                elif mod_mass == 79.9568:
                    if add_aa in 'STY':
                        add_aa = 'STY'
                elif mod_mass == 31.9898:
                    if add_aa in 'MW':
                        add_aa = 'MW'
            if mod_mass == -18.0106:
                if add_aa == 'E':
                    modifs_posit[posit] = modif_convers_dict[mod_mass][1].format(add_aa)
                else:
                    if add_aa in 'ST':
                        add_aa = 'ST'  
                    modifs_posit[posit] = modif_convers_dict[mod_mass][0].format(add_aa)
            else:    
                modifs_posit[posit] = modif_convers_dict[mod_mass].format(add_aa)
        
        modif_sequence = ''.join(["".join(i) for i in zip(' '+ sequence, modifs_posit)]).strip()
        return modif_sequence
    
    else: 
        return sequence

In [ ]:
#hide

def test_convert_fragpipe_mq_mod():
    seq1 = 'AAEREPPPLGDGKPTDFEDLEDGEDLFTSTVSTLE'
    modif1 = 'N-term(42.0106)'
    assert '[Acetyl (N-term)]AAEREPPPLGDGKPTDFEDLEDGEDLFTSTVSTLE' == convert_fragpipe_mq_mod(seq1, modif1)
    seq2 = 'AAEVISDARENIQRFFGHGAEDSLADQAANEWGRSGKDPNHFRPAGLPEKY'
    modif2 = 'C-term(-0.9840)'
    assert 'AAEVISDARENIQRFFGHGAEDSLADQAANEWGRSGKDPNHFRPAGLPEKY[Amidated (C-term)]' == convert_fragpipe_mq_mod(seq2, modif2)
    seq3 = 'QESQSEEIDCNDKDLFKA'
    modif3 = '1Q(-17.0265)'
    assert 'Q[Gln->pyro-Glu]ESQSEEIDCNDKDLFKA' == convert_fragpipe_mq_mod(seq3, modif3)
    seq4 = 'EKPLLEKSHCIAEVENDEMPA'
    modif4 = '1E(-18.0106)'
    assert 'E[Glu->pyro-Glu]KPLLEKSHCIAEVENDEMPA' == convert_fragpipe_mq_mod(seq4, modif4)
    seq5 = 'SKPLLEKSHCIAEVENDEMPA'
    modif5 = '1S(-18.0106)'
    assert 'S[Dehydrated (ST)]KPLLEKSHCIAEVENDEMPA' == convert_fragpipe_mq_mod(seq5, modif5)
    seq6 = 'YKPLLEKSHCIAEVENDEMPA'
    modif6 = '1Y(-18.0106)'
    assert 'Y[Dehydrated (Y)]KPLLEKSHCIAEVENDEMPA' == convert_fragpipe_mq_mod(seq6, modif6)
    
    seq1_several_dif_mods = 'AAAAECDVVMAATEPELLDDQEAK'
    seq1_several_dif_mods_modifs = '10M(15.9949),6C(57.0215),N-term(42.0106)'
    assert '[Acetyl (N-term)]AAAAEC[Carbamidomethyl (C)]DVVM[Oxidation (M)]AATEPELLDDQEAK' == convert_fragpipe_mq_mod(
        seq1_several_dif_mods, seq1_several_dif_mods_modifs)
    seq2_several_dif_mods = 'PGFSIADKKR'
    seq2_several_dif_mods_modifs = '8K(114.0429),9K(114.0429)'
    assert 'PGFSIADK[GlyGly (K)]K[GlyGly (K)]R' == convert_fragpipe_mq_mod(
        seq2_several_dif_mods, seq2_several_dif_mods_modifs)
    
    seq2_several_same_mods = 'AAAAAAVSGSAAAEAKECDRPCVN'
    seq2_several_same_mods_modifs = '18C(57.0215), 22C(57.0215)'
    assert 'AAAAAAVSGSAAAEAKEC[Carbamidomethyl (C)]DRPC[Carbamidomethyl (C)]VN' == convert_fragpipe_mq_mod(
        seq2_several_same_mods, seq2_several_same_mods_modifs)
    seq3_several_same_mods = 'AAETVPDLPSPPTEAPAPASNTSTR'
    seq3_several_same_mods_modifs = '10S(79.9663),20S(79.9663),22T(79.9663),23S(79.9663)'
    assert 'AAETVPDLPS[Phospho (STY)]PPTEAPAPAS[Phospho (STY)]NT[Phospho (STY)]S[Phospho (STY)]TR' == convert_fragpipe_mq_mod(
        seq3_several_same_mods, seq3_several_same_mods_modifs)
    
    seq_no_mod = 'CVNTTLQIK'
    seq_no_mod_modifs = ''
    assert "CVNTTLQIK" == convert_fragpipe_mq_mod(seq_no_mod, seq_no_mod_modifs)
    
test_convert_fragpipe_mq_mod()

## Aggregated import function

In [ ]:
#export
import pandas as pd
import re
from io import StringIO
import os

def import_data(
    file :str,
    sample: Union[str, list, None] = None,
    verbose: bool = True,
    dashboard: bool = False
) -> pd.DataFrame:
    """Import peptide level data. Depending on available columns in the provided file, the function calls other specific functions for each tool.

    Args:
        file (str): The name of a file.
        sample (Union[str, list, None]): The unique raw file name(s) to filter the original file. Defaults to None. In this case data for all raw files will be extracted.
        verbose (bool): if True, print the type of input that is used. Defaults to True.
        dashboard (bool): If True, the function is used for the dashboard output (StringIO object). Defaults to False.

    Raises:
        TypeError: If the input data format is unknown.

    Returns:
        pd.DataFrame: A pandas dataframe containing information about: all_protein_ids (str), modified_sequence (str), naked_sequence (str)
    """
    if dashboard:
        file = StringIO(str(file, "utf-8"))

    file_ext = os.path.splitext(file)[-1]
    if file_ext=='.csv':
        sep=','
    elif file_ext=='.tsv':
        sep='\t'
    elif file_ext=='.txt':
        sep='\t'

    with open(file) as filelines:
        i = 0
        pos = 0
        for l in filelines:
            i += 1
            l = l.split(sep)
            if i>0:
                break

        uploaded_data_columns = set(l)
        input_info = file
    if set(["Proteins","Modified sequence","Raw file"]).issubset(uploaded_data_columns):
        if verbose:
            print("Import MaxQuant input")
        data = import_maxquant_data(input_info, sample=sample)
    elif set(["PEP.AllOccurringProteinAccessions","EG.ModifiedSequence","R.FileName"]).issubset(uploaded_data_columns):
        if verbose:
            print("Import Spectronaut input")
        data = import_spectronaut_data(input_info, sample=sample)
    elif set(["protein_group", "sequence", "shortname"]).issubset(uploaded_data_columns):
        if verbose:
            print("Import AlphaPept input")
        data = import_alphapept_data(input_info, sample=sample)
    elif set(["Protein.Ids", "Modified.Sequence", "Run"]).issubset(uploaded_data_columns):
        if verbose:
            print("Import DIA-NN input")
        data = import_diann_data(input_info, sample=sample)
    else:
        raise TypeError(f'Input data format for {file} not known.')
    return data

In [ ]:
#hide
import sys

def test_import_data():
    data_MQ = import_data("../testdata/test_maxquant_input.txt", verbose=False)
    test = pd.read_csv('../testdata/test_maxquant_imported.csv', sep=',') 
    pd.testing.assert_frame_equal(data_MQ, test)
    
    data_S_csv = import_data("../testdata/test_spectronaut_input.csv", verbose=False)
    data_S_tsv = import_data("../testdata/test_spectronaut_input.tsv", verbose=False)
    pd.testing.assert_frame_equal(data_S_csv, data_S_tsv)
    test = pd.read_csv('../testdata/test_spectronaut_imported.csv', sep=',') 
    pd.testing.assert_frame_equal(data_S_csv, test)
    
    data_S_sub = import_data("../testdata/test_spectronaut_input.csv", 
                             sample = "raw_01", 
                             verbose=False)
    assert data_S_sub.shape[0] == 40
    
    data_alphapept = import_data("../testdata/test_alphapept_input.csv", 
                             sample = "exp_1", 
                             verbose=False)
    assert data_alphapept.shape[0] == 2127
    
    data_diann = import_data("../testdata/test_diann_input.tsv", 
                                 sample = "20201218_tims03_Evo03_PS_SA_HeLa_200ng_high_speed_21min_8cm_S2-B2_1_22648", 
                                 verbose=False)
    assert data_diann.shape[0] == 39
    
    try:
        out = import_data("../testdata/test_uniprot_df.csv")
    except TypeError as e:
        out = e
    assert str(out) == "Input data format for ../testdata/test_uniprot_df.csv not known."    

test_import_data()

In [ ]:
#hide
### Export notebook to script

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted Importing.ipynb.
Converted Preprocessing.ipynb.
Converted SequencePlot.ipynb.
Converted Uniprot_integration.ipynb.
Converted index.ipynb.
Converted organisms_data.ipynb.
Converted proteolytic_cleavage.ipynb.
